In [77]:
import pandas as pd 
import numpy as np 
import re
from openai import OpenAI
import ast
from tqdm import tqdm 
client = OpenAI(api_key=open("/Users/mosobay/Dropbox (MIT)/research/openai_api", "r").read().strip())

In [2]:
def clean_task_html(text):
    pattern = r"(?<=Set-Up)(.*)"
    match = re.search(pattern, text)
    extracted_text = match.group(1)
    no_html = re.sub(r"<[^>]+>", "", extracted_text)
    return no_html 

In [3]:
# df_tasks = pd.read_csv("102_tasks_with_sources_clean.csv")
df_tasks = pd.read_csv("/Users/mosobay/Dropbox (MIT)/research/homo_silicus/task_htmls.csv")
df_tasks["task_desc"] = [clean_task_html(x) for x in df_tasks["html"]]
df_abilities = pd.read_excel("/Users/mosobay/Downloads/Abilities.xlsx")

In [73]:
df_task_sample = df_tasks.sample(10, random_state=2024)

In [32]:
skill_str = """Active Learning
Active Listening
Complex Problem Solving
Coordination
Critical Thinking
Instructing Judgment and Decision Making
Learning Strategies
Management of Financial Resources
Management of Material Resources
Management of Personnel Resources
Mathematics
Monitoring
Negotiation
Operations Analysis
Persuasion
Programming
Reading Comprehension
Science
Service Orientation
Social Perceptiveness
Speaking
Systems Analysis
Systems Evaluation
Time Management
Writing
Category Flexibility
Deductive Reasoning
Fluency of Ideas
Inductive Reasoning
Information Ordering
Mathematical Reasoning
Memorization
Near Vision
Number Facility
Oral Comprehension
Oral Expression
Originality
Problem Sensitivity
Speech Clarity
Speech Recognition
Speed of Closure
Written Comprehension
Written Expression
Administration and Management
Biology
Clerical
Communications and Media
Computers and Electronics
Customer and Personal Service
Economics and Accounting
Education and Training
English Language
Fine Arts
Foreign Language
Geography
History and Archeology
Law and Government
Mathematics Knowledge
Medicine and Dentistry
Personnel and Human Resources
Philosophy and Theology
Psychology
Sales and Marketing
Sociology and Anthropology
Telecommunications
Therapy and Counseling
Analyzing Data or Information
Assisting and Caring for Others
Coaching and Developing Others
Communicating with Persons Outside Organization
Communicating with Supervisors, Peers, or Subordinates
Coordinating the Work and Activities of Others
Developing Objectives and Strategies
Developing and Building Teams
Documenting/Recording Information
Establishing and Maintaining Interpersonal Relationships
Evaluating Information to Determine Compliance with Standards
Getting Information
Guiding, Directing, and Motivating Subordinates
Identifying Objects, Actions, and Events
Interacting With Computers
Interpreting the Meaning of Information for Others
Judging the Qualities of Things, Services, or People
Making Decisions and Solving Problems
Monitoring and Controlling Resources
Organizing, Planning, and Prioritizing Work
Performing Administrative Activities
Performing for or Working Directly with the Public
Processing Information
Provide Consultation and Advice to Others
Resolving Conflicts and Negotiating with Others
Scheduling Work and Activities
Selling or Influencing Others
Staffing Organizational Units
Thinking Creatively
Training and Teaching Others
Updating and Using Relevant Knowledge"""

In [39]:
list_cog_skills = skill_str.split("\n")
list_cog_skills = [f"SKILL ID {x}: {list_cog_skills[x]}" for x in range(len(list_cog_skills))]

In [75]:
def get_gpt_sorted_skills(task_desc):
    system_prompt = """You are assisting a manager in understanding the relative importance of different abilities to different tasks. The tasks are all completed on a screen.
    You will be presented with a task description, and then a list of 97 skills. Return the COMPLETE list of skills, sorted in descending order of their importance to successfully completing the task.
    ONLY RETURN THE LIST OF SKILL IDS IN ORDER OF IMPORTANCE TO THE TASK, WITHOUT ANY OTHER TEXT. 
    RETURN THE LIST IN THE FORMAT [LIST ITEM 1, LIST ITEM 2, ...]"""
    
    task_prompt = f"The task description: {task_desc}"
    
    skill_prompt = "The list of skills is:\n" + "\n".join(list_cog_skills)
    
    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": task_prompt},
            {"role": "user", "content": skill_prompt}])
    
    return response
    

In [78]:
gpt_responses = []

for task_desc in tqdm(df_task_sample["task_desc"].values): 
    gpt_responses.append(get_gpt_sorted_skills(task_desc))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [02:39<00:00, 15.97s/it]


In [80]:
df_task_sample["gpt_response"] = gpt_responses
df_task_sample["gpt_parsed"] = [ast.literal_eval(response.choices[0].message.content) for response in df_task_sample["gpt_response"]]

In [92]:
df_task_sample.loc[:,list_cog_skills] = pd.DataFrame([dict(zip(np.array(list_cog_skills)[x], range(len(x)))) for x in df_task_sample["gpt_parsed"]]).to_numpy()

In [97]:
df_task_sample[list_cog_skills].describe().T.sort_values("mean")[:50]

,count,mean,std,min,25%,50%,75%,max
SKILL ID 4: Critical Thinking,10.0,9.600000,4.880801,4.0,6.00,8.0,12.75,20.0
SKILL ID 5: Instructing Judgment and Decision Making,10.0,10.600000,10.178409,1.0,1.50,8.5,16.50,31.0
SKILL ID 20: Speaking,10.0,12.200000,9.863513,2.0,5.00,9.0,19.00,32.0
SKILL ID 21: Systems Analysis,10.0,13.300000,9.730251,4.0,6.00,9.5,20.00,33.0
SKILL ID 34: Oral Comprehension,10.0,15.400000,14.803903,0.0,4.25,10.5,21.75,45.0
SKILL ID 6: Learning Strategies,10.0,16.600000,11.197222,0.0,6.25,18.5,26.00,32.0
SKILL ID 7: Management of Financial Resources,10.0,16.900000,10.268073,2.0,7.25,18.5,26.25,29.0
SKILL ID 0: Active Learning,10.0,17.900000,10.959015,0.0,10.25,18.5,23.00,34.0
SKILL ID 28: Inductive Reasoning,10.0,18.100000,6.261878,8.0,14.75,17.5,22.75,28.0
SKILL ID 57: Mathematics Knowledge,10.0,18.600000,19.184484,1.0,2.25,15.0,27.00,57.0


In [102]:
df_task_sample.sort_values("SKILL ID 55: History and Archeology")[["task", "SKILL ID 55: History and Archeology"]]

,task,SKILL ID 55: History and Archeology
26,Typing game,8.0
14,Letters-to-numbers problems (cryptography),24.0
89,Volunteer Investment Game,37.0
17,Advertisement writing,39.0
87,Apache helicopter flight simulator (Longbow2),43.0
97,Investment game (hidden-profile),55.0
72,Arithmetic problem 2,65.0
49,Desert survival,74.0
19,Putting food into categories,79.0
34,Checkers,92.0


In [115]:
df_task_sample.loc[34, list_cog_skills].sort_values()[:50]

SKILL ID 27: Fluency of Ideas                                             0.0
SKILL ID 5: Instructing Judgment and Decision Making                      1.0
SKILL ID 57: Mathematics Knowledge                                        2.0
SKILL ID 20: Speaking                                                     3.0
SKILL ID 21: Systems Analysis                                             4.0
SKILL ID 22: Systems Evaluation                                           5.0
SKILL ID 4: Critical Thinking                                             6.0
SKILL ID 92: Selling or Influencing Others                                7.0
SKILL ID 34: Oral Comprehension                                           8.0
SKILL ID 29: Information Ordering                                         9.0
SKILL ID 25: Category Flexibility                                        10.0
SKILL ID 31: Memorization                                                11.0
SKILL ID 52: Fine Arts                                          

In [103]:
df_task_sample["task_desc"][26]

'Participants see an image of a document &mdash;  we can call this the "Document to Copy." This document contains several passages of relatively complicated text.The Document to Copy cannot be edited, copied, or pasted. Everyone working together sees the same document.Participants also have a shared, editable document, where they are able to type.2. Objective / GoalThe goal is to exactly reproduce the words from the Document to Copy in the editable document. The focus is on typing words correctly: participants earn points for every word they correctly type, and they lose points for skipping words or for making typos.Participants have a limited amount of time to type as many words from the Document to Copy as they can.If working in a group, everyone types into the same shared document, and therefore team members must also coordinate to avoid typing over each other or missing certain sections.'

In [ ]:
print(response.choices[0].message.content)

[2, 10, 30, 57, 26, 28, 29, 66, 80, 83, 21, 22, 32, 6, 0, 4, 37, 94, 36, 31, 25, 13, 27, 81, 77, 33, 11, 85, 23, 41, 16, 42, 24, 34, 35, 5, 82, 68, 75, 73, 1, 3, 70, 20, 69, 87, 14, 19, 50, 61, 65, 89, 88, 47, 15, 76, 84, 91, 40, 38, 39, 7, 8, 9, 43, 59, 72, 86, 93, 78, 71, 17, 44, 51, 52, 54, 55, 56, 58, 60, 62, 63, 64, 45, 48, 49, 53, 90, 92, 95, 96]


array(['SKILL ID 2: Complex Problem Solving', 'SKILL ID 10: Mathematics',
       'SKILL ID 30: Mathematical Reasoning',
       'SKILL ID 57: Mathematics Knowledge',
       'SKILL ID 26: Deductive Reasoning',
       'SKILL ID 28: Inductive Reasoning',
       'SKILL ID 29: Information Ordering',
       'SKILL ID 66: Analyzing Data or Information',
       'SKILL ID 80: Interacting With Computers',
       'SKILL ID 83: Making Decisions and Solving Problems',
       'SKILL ID 21: Systems Analysis', 'SKILL ID 22: Systems Evaluation',
       'SKILL ID 32: Near Vision', 'SKILL ID 6: Learning Strategies',
       'SKILL ID 0: Active Learning', 'SKILL ID 4: Critical Thinking',
       'SKILL ID 37: Problem Sensitivity',
       'SKILL ID 94: Thinking Creatively', 'SKILL ID 36: Originality',
       'SKILL ID 31: Memorization', 'SKILL ID 25: Category Flexibility',
       'SKILL ID 13: Operations Analysis',
       'SKILL ID 27: Fluency of Ideas',
       'SKILL ID 81: Interpreting the Meaning of Inform

In [71]:
np.array(list_cog_skills)[ast.literal_eval(response.choices[0].message.content)]

array(['SKILL ID 10: Mathematics', 'SKILL ID 57: Mathematics Knowledge',
       'SKILL ID 30: Mathematical Reasoning',
       'SKILL ID 33: Number Facility', 'SKILL ID 26: Deductive Reasoning',
       'SKILL ID 28: Inductive Reasoning',
       'SKILL ID 4: Critical Thinking', 'SKILL ID 54: Geography',
       'SKILL ID 29: Information Ordering',
       'SKILL ID 66: Analyzing Data or Information',
       'SKILL ID 83: Making Decisions and Solving Problems',
       'SKILL ID 77: Getting Information',
       'SKILL ID 80: Interacting With Computers',
       'SKILL ID 16: Reading Comprehension',
       'SKILL ID 41: Written Comprehension',
       'SKILL ID 42: Written Expression', 'SKILL ID 0: Active Learning',
       'SKILL ID 6: Learning Strategies', 'SKILL ID 23: Time Management',
       'SKILL ID 17: Science', 'SKILL ID 25: Category Flexibility',
       'SKILL ID 36: Originality', 'SKILL ID 94: Thinking Creatively',
       'SKILL ID 1: Active Listening', 'SKILL ID 34: Oral Comprehensio

In [63]:
len(ast.literal_eval(response.choices[0].message.content))

91

In [55]:
print(response.choices[0].message.content)

For the task of finding the shortest path on a map (akin to the "traveling salesman" problem), here is the complete list of skills, sorted in descending order of their importance to successfully completing the task:

1. SKILL ID 10: Mathematics
2. SKILL ID 2: Complex Problem Solving
3. SKILL ID 13: Operations Analysis
4. SKILL ID 21: Systems Analysis
5. SKILL ID 22: Systems Evaluation
6. SKILL ID 83: Making Decisions and Solving Problems
7. SKILL ID 94: Thinking Creatively
8. SKILL ID 66: Analyzing Data or Information
9. SKILL ID 4: Critical Thinking
10. SKILL ID 85: Organizing, Planning, and Prioritizing Work
11. SKILL ID 23: Time Management
12. SKILL ID 30: Mathematical Reasoning
13. SKILL ID 57: Mathematics Knowledge
14. SKILL ID 88: Processing Information
15. SKILL ID 79: Identifying Objects, Actions, and Events
16. SKILL ID 5: Instructing Judgment and Decision Making
17. SKILL ID 96: Updating and Using Relevant Knowledge
18. SKILL ID 76: Evaluating Information to Determine Complia